In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['logs', 'Window timings and IBI.ipynb', 'feature_development.ipynb', 'data', 'data_and_processing_description.ipynb', 'featured_eeg_prediction.ipynb', 'featured_prediction_random_forest.ipynb', 'gait_data_exploration.ipynb', 'featured_eeg_prediction_gradient_boosting.ipynb', 'Analyse_results.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'featured_E4_prediction.ipynb', 'data_investigation_scratch.ipynb', 'grouping_and_crossvalidation.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")


thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH


from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.windowed import get_window_start_in_event_time
from predicament.data.windowed import get_window_duration
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import MAXIMAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from prepare_evaluation_data import load_dataframe_and_config


In [5]:
from predicament.data.events import load_event_info_from_csv


In [6]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [7]:
subdir = '20231206193533'
windowed_data_dir = os.path.join(WINDOWED_BASE_PATH,subdir)

windoweddf, windowed_config = load_dataframe_and_config(
    windowed_data_dir, 'windowed.csv')

In [8]:
n_channels = int(windowed_config['LOAD']['n_channels'])
channels = json.loads(windowed_config['LOAD']['channels'].replace("'",'"'))
participant_list = json.loads(windowed_config['LOAD']['participant_list'].replace("'",'"'))
conditions = json.loads(windowed_config['LOAD']['conditions'].replace("'",'"'))
sample_rate = int(windowed_config['LOAD']['sample_rate'])
window_size = int(windowed_config['LOAD']['window_size'])
window_step = int(windowed_config['LOAD']['window_step'])
time = window_size/sample_rate
print(f"sample_rate: {sample_rate}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")
window_duration = get_window_duration(window_size, sample_rate)

print(f"window_size: {window_size}, window_step: {window_step}, window_duration: {window_duration}")

sample_rate: 64, n_samples = 256, time: 4.0s, n_channels: 7
window_size: 256, window_step: 32, window_duration: 4.0


In [9]:
all_participants_events = load_event_info_from_csv(
        participant_list)

VG_01:setup is valid but timestamps have issues. What does valid mean?
VG_01:baseline is valid but timestamps have issues. What does valid mean?
VG_06:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VG_07:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VG_08:family_inter is valid but timestamps have issues. What does valid mean?
VG_10:setup is valid but timestamps have issues. What does valid mean?
VG_10:takeoff_EEG is valid but timestamps have issues. What does valid mean?
VH_01:family_inter is valid but timestamps have issues. What does valid mean?


In [10]:
participant = 'VG_01'
condition_id = 1
condition_name = conditions[0]
participant_events = all_participants_events[participant]
filter_ = (windoweddf['participant']==participant)&(windoweddf['condition']==condition_id)
n_windows = len(windoweddf[filter_])
windoweddf.loc[filter_, ('window index',)] = np.arange(n_windows)
window_start_unixtimes = np.empty(n_windows)
for w, window_index in enumerate(windoweddf[filter_]['window index']):
    window_start_in_event_time = \
        get_window_start_in_event_time(
            window_index, window_step, sample_rate)
    window_start_unixtime = \
        participant_events.in_event_time_to_unixtime(
            condition_name,window_start_in_event_time)
    window_start_unixtimes[w] = window_start_unixtime
window_start_unixtimes

array([1.63111335e+09, 1.63111335e+09, 1.63111335e+09, 1.63111335e+09,
       1.63111335e+09, 1.63111335e+09, 1.63111335e+09, 1.63111335e+09,
       1.63111335e+09, 1.63111335e+09, 1.63111336e+09, 1.63111336e+09,
       1.63111336e+09, 1.63111336e+09, 1.63111336e+09, 1.63111336e+09,
       1.63111336e+09, 1.63111336e+09, 1.63111336e+09, 1.63111336e+09,
       1.63111336e+09, 1.63111336e+09, 1.63111336e+09, 1.63111336e+09,
       1.63111336e+09, 1.63111336e+09, 1.63111336e+09, 1.63111336e+09,
       1.63111336e+09, 1.63111336e+09, 1.63111336e+09, 1.63111337e+09,
       1.63111337e+09, 1.63111337e+09, 1.63111337e+09, 1.63111337e+09,
       1.63111337e+09, 1.63111337e+09, 1.63111337e+09, 1.63111337e+09,
       1.63111337e+09, 1.63111337e+09, 1.63111337e+09, 1.63111337e+09,
       1.63111337e+09, 1.63111337e+09, 1.63111337e+09, 1.63111337e+09,
       1.63111337e+09, 1.63111337e+09, 1.63111338e+09, 1.63111338e+09,
       1.63111338e+09, 1.63111338e+09, 1.63111338e+09, 1.63111338e+09,
      

In [14]:
window_start_unixtimes[1:11] - window_start_unixtimes[0:10]

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [11]:
len(windoweddf)

25296

In [12]:
# subdir = '20231206193533'

# featured_data_dir = os.path.join(FEATURED_BASE_PATH,subdir)

# featureddf, featured_config = load_dataframe_and_config(
#     featured_data_dir, 'featured.csv')
# n_channels = int(featured_config['LOAD']['n_channels'])
# channels = json.loads(featured_config['LOAD']['channels'].replace("'",'"'))
# participant_list = json.loads(featured_config['LOAD']['participant_list'].replace("'",'"'))
# Fs = int(featured_config['LOAD']['sample_rate'])
# window_size = int(featured_config['LOAD']['window_size'])
# time = window_size/Fs
# print(f"Fs: {Fs}, n_samples = {window_size}, time: {time}s, n_channels: {n_channels}")
